## Preparations

После перезагрузки компьютера, запустите следующие команды в терминале (пароль 1qaz!QAZ).
```bash
echo 1024 | sudo tee /sys/module/usbcore/parameters/usbfs_memory_mb
sudo cat /sys/module/usbcore/parameters/usbfs_memory_mb
```

В трех отдельных сессиях терминала запустите, если еще не запущено, по одной строке из
```bash
bash ~/Downloads/dev.sk_robot_rgbd_data/src/skrgbd/scanning/utils/log_viewer.sh all
bash ~/Downloads/dev.sk_robot_rgbd_data/src/skrgbd/scanning/utils/log_viewer.sh info
bash ~/Downloads/dev.sk_robot_rgbd_data/src/skrgbd/scanning/utils/log_viewer.sh err
```
В этих сессиях будут отображаться (1) все сообщения с информацией о сканировании, (2) сообщения об основных этапах сканирования, (3) ошибки.

## Start the cameras

1. Запустите код в следующей клетке: он инициализирует камеры.
Проверьте по логам что все 6 камер инициализировались корректно.

In [ ]:
from pathlib import Path
import datetime
import sys

sys.path.append('/home/universal/Downloads/dev.sk_robot_rgbd_data/src')
from skrgbd.scanning.scan_helper.scan_helper import ScanHelper


scans_root = '/mnt/data/sk3d/raw_scans'
today = datetime.datetime.now().strftime('%y_%m_%d')
logfile = f'/home/universal/Downloads/dev.sk_robot_rgbd_data/experiments/logs/scanning/{today}_cameras.log'

scan_helper = ScanHelper(scans_root, logfile, trajectories='tabletop', thread='cameras')
scan_helper.stop_warmup()

## Set the white balance of the cameras
Настройте баланс белого один раз после запуска тетрадки.
1. Выключите свет в комнате.
2. Запустите код в следующей клетке и следуйте инструкциям.

In [ ]:
robot = scan_helper.camera_trajectory_class()
robot.move_for_awb(.1)
scan_helper.set_white_balance()
scan_helper.stop_warmup()
robot.rest(.1)
del robot

## Pick settings

Проверьте правильность расположения ламп, и с помощью процедуры ниже подберите настройки камер.

In [ ]:
scene_name = 'test_tabletop'

light_setups = [
    ('flash', 'best'),
    ('flash', 'fast'),
    ('soft_left', 'best'),
    ('soft_right', 'best'),
    ('soft_top', 'best'),
    ('hard_left_bottom_far', 'best'),
    ('hard_right_top_far', 'best'),
    ('hard_right_top_close', 'best'),
    ('hard_left_top_far', 'best'),
    ('hard_right_bottom_close', 'best'),
    ('ambient', 'best'),
    ('ambient_low', 'fast'),
]

scan_helper.pick_camera_settings(scene_name, light_setups)
scan_helper.stop_warmup()

## Stop the cameras

1. Для остановки камер в конце смены или перед перезапуском тетрадки запустите код ниже.
Если клетка не завершается в течение 10 секунд, и камеры продолжают снимать, то, скорее всего, клетка зависла --- смотрите шаги 2-4.
Если клетка не зависла, переходите к шагу 5.

In [ ]:
scan_helper.stop()

Если на шаге 1 установка зависла

2. Перезапустите ядро тетрадки.
3. Запустите код ниже.
4. Если клетка не завершается в течение 10 секунд, повторите с шага 2.

In [ ]:
import datetime
import sys

sys.path.append('/home/universal/Downloads/dev.sk_robot_rgbd_data/src')
from skrgbd.scanning.scan_helper.scan_helper import ScanHelper

scans_root = '/mnt/data/sk3d/raw_scans'
today = datetime.datetime.now().strftime('%y_%m_%d')
logfile = f'/home/universal/Downloads/dev.sk_robot_rgbd_data/experiments/logs/scanning/{today}_cameras.log'
scan_helper = ScanHelper(scans_root, logfile, trajectories='tabletop', thread='cameras')
scan_helper.stop()

5. Перезапустите ядро тетрадки.
6. Если робот не находится в стартовом положении, переведите его в стартовое положение как показано в видеоинструкции.
7. Выключите робота через пульт.
8. Выключите сетевой фильтр который питает лампы, и выдерните его из розетки.
9. Проверьте что экраны телефонов погасли. Если нет --- выключите телефоны кнопкой.